<a href="https://colab.research.google.com/github/busrasencan/NeuralMachineTranslation/blob/master/NMT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **NMT**

NMT, doğal dil işlemedeki en önemli alanlardan birisidir.

NMT'nin iki parçaya ihtiyacı olduğu için iyi bir test aracıdır.
- Dil analizi ve dil modelleme.

Günlük hayatta, ticartte, diplomaside, askeride makine çevirisine çok büyük ihtiyaç var.

- Yıllık 40 milyar dolarlık endüstri.

Google 2016'da Google Translate'te makine çevirisine geçti. Günlük 100 milyardan fazla kelime çeviriyor.

Facebook 2016'da kendi makine çevirisi modelini geliştirdi.

Ebay sınır ötesi ticaret için makine çevirisine bağımlı.

Çeviri yapabilmek için uçtan uca eğitebileceğimiz büyükçe tek bir yapay sinir ağı oluşturmaya neural machine translation denir.

Bu çalışmada seq2seq kullanacağız. Hadi başlayalım.


In [0]:
pip install tensorflow==2.0.0-rc0

In [0]:
import tensorflow as tf
import numpy as np

In [0]:
tf.__version__

'2.0.0-rc0'

In [0]:
tf.keras.__version__

'2.2.4-tf'

In [0]:
import matplotlib.pyplot as plt
import math
import os

In [0]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, GRU, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [0]:
mark_start = 'ssss '
mark_end = ' eeee'

Model çeviri yaparken decoder ile cümle üretecek ve çeviri gerçekleştirecek. Cümle üretmeye başlaması için başlangıç tokeni veriyoruz. Bu işlem için kelime haznesinde bulunmayan bir başlangıç tokeni üretiyoruz. Cümlenin bir noktada sonlanması gerekir bunun içinde bir token daha belirleyeceğiz ve sonlandırma tokeni olacak. Burada boşluklar çok önemli tokenlerin kelimeye yapışmasını istemeyiz. Bu sebeple başlangıç tokeninin sonuna, bitiş tokeninin ise başına boşluk koyuyoruz.

In [0]:
data_serc = []
data_dest = []


Şimdi ise encoder ve decoder için iki liste oluşturuyoruz. Encoder'a inglizce cümle decoder'a ise türkçe cümleleri vereceğiz.

In [0]:
for line in open('/content/drive/My Drive/NLP/NLP/5- Makine Çevirisi/tur.txt', encoding = 'UTF-8'):
  en_text, tr_text = line.rstrip().split ('\t')
  
  tr_text = mark_start + tr_text + mark_end

  data_serc.append(en_text)
  data_dest.append(tr_text)

Artık verileri alıp listeye ekleyebiliriz. Veriler tur.txt dosyasının içerisinde yer alıyor. Veri dosyasında her satırda ingilizce cümleler ve türkçe çevirileri var. For döngüsüyle satır satır geçerek ingilizce cümleyi ve türkçe çevirisini alacağız. Daha sonra döngünün içinde yer alan veriyi ingilizce, türkçe olarak ayırıp iki farklı listeye yazacağız. Encoding utf-8 yapmazsak türkçe karakterlerde sıkıntılar yaşayabiliriz. Veri setinde inglizce ve türkçe cümleler tab ile ayrıldığı için split ile veriyi ikiye ayırıyoruz. Rstrip kodu ise satırdaki gereksiz boşluk karakterlerini silecektir. Şimdi tr_text'e başlangıç ve bitiş tokenini ekliyoruz. Cümleleri listeye eklemeye hazırız. Verinin bazı elemanlarına bakalım.

In [0]:
idx = 200
data_serc[idx]

'How deep?'

In [0]:
data_dest[idx]

'ssss Ne kadar derin? eeee'

In [0]:
len(data_serc)

473035

In [0]:
len(data_dest)

473035

Her iki veride de toplamda 473035 tane cümle bulunmaktadır. Şimdi tokenleştirme yapalım. bu işlem için bir sınıf yazacağız. İşimizi kolaylaştırmak için tokenizer'a özellikler ekleyeceğiz. Tokenleştirme için adımlar sınıfta toplanacak bu sayede encoder ve decoder kolay bir şekilde tokenleştirilcek. 

In [0]:
num_words = 10000

In [0]:
class TokenizerWrap(Tokenizer):
    def __init__(self, texts, padding,
                 reverse=False, num_words=None):
       
        Tokenizer.__init__(self, num_words=num_words)

        self.fit_on_texts(texts)

        self.index_to_word = dict(zip(self.word_index.values(),
                                      self.word_index.keys()))

        self.tokens = self.texts_to_sequences(texts)

        if reverse:
            self.tokens = [list(reversed(x)) for x in self.tokens]
            truncating = 'pre'
        else:
            truncating = 'post'

        self.num_tokens = [len(x) for x in self.tokens]
        self.max_tokens = np.mean(self.num_tokens) \
                          + 2 * np.std(self.num_tokens)
        self.max_tokens = int(self.max_tokens)


        self.tokens_padded = pad_sequences(self.tokens,
                                           maxlen=self.max_tokens,
                                           padding=padding,
                                           truncating=truncating)

    def token_to_word(self, token):
        word = " " if token == 0 else self.index_to_word[token]
        return word 

    def tokens_to_string(self, tokens):
        words = [self.index_to_word[token]
                 for token in tokens
                 if token != 0]
        text = " ".join(words)
        return text
    
    def text_to_tokens(self, text, reverse=False, padding=False):
        tokens = self.texts_to_sequences([text])
        tokens = np.array(tokens)

        if reverse:
            tokens = np.flip(tokens, axis=1)
            truncating = 'pre'
        else:
            truncating = 'post'

        if padding:
            tokens = pad_sequences(tokens,
                                   maxlen=self.max_tokens,
                                   padding='pre',
                                   truncating=truncating)

        return tokens

Tokenizerwrap çağırıldığı zaman python bir nesne oluşturur ve ilk nesneyi inite verir. Buraya ilk olarak hep self verilir, nesnenin kendisidir. Daha sonra sınıf içerisine alacağımız parametreler verilir. Önce text ile tokenleştirme işlemi yapacağımız için text'i alıyoruz. Daha sonra padding ile tokenlerin başına mı sonuna mı padding eklenecek belirteceğiz. Son parametremiz olan num_words ile kelime haznesinde kaç kelime olacağını belirtiyoruz. Şimdi tokenleştirme işlemlerini yazalım. Aldığımız texti tokenleştiriyoruz. Word_index ile anahtar ve değerlerin yerini değiştireceğiz. Normalde word_indeks içerisinde anahtar kelimelerden, değerler ise kelimeleri sembolize eden değerlerden oluşuyor. Biz burada sayıları anahtar yapıp değerleri kelimeler yapacağız ve bunu bir sözlükte (dict komutu) kaydedeceğiz.
 
Bu şekilde elimizdeki sayılar hangi kelimeye karşılık geliyor bulabilir, tokenleri tekrar stringe dönüştürebiliriz. Şimdi sözlük için dict komutunu kullanıyoruz ve zip'in içerisine önce anahtarları sonra değerleri veriyoruz. Values içinde kelimelere karşılık gelen sayılar var bu sayıları anahtar yapıyoruz. Daha sonra kelimeleri alıyoruz.

Elimizdeki yazıları tokenler olarak bir listede topluyoruz. Tüm inputları aynı boyuta getirmek için padding ekliyoruz. Başa token eklemeye pre padding, sona token eklemeye ise post padding deniyor. Pading yaparken boyut belirlendiğinde boyuttan büyük olanı kısaltıyorduk bu işleme ise trancating deniyor ve bu da padding gibi iki farklı yönteme sahiptir. Pre derken baştan, post derken sondan kısaltılır.

Çeviri yaparken hem pre hem post kullanılır. Encoder için pre, decoder için post uygulayacağız. Reverse içerisinde koşula göre trancating true ise pre, false ise post olacak. Cümle başı bizim için önemli olduğu için cümleyi ters çevireceğiz. Tüm kelimelerin yerini değiştireceğiz. Bu işlemi ise reversed ile gerçekleştirdik.

Max_tokens büyüdükçe eğitim hızı düşecektir. Artık cümleyi tokenleştirdik ve tokenleştirme işlemi yaptık. Sınıf sayesinde hızlı bir şekilde encoder ve decoder için tokenleştirme gerçekleştirebiliriz.

In [0]:
tokenizer_src = TokenizerWrap(texts=data_serc,
                              padding='pre',
                              reverse=True,
                              num_words=num_words)

In [0]:
tokenizer_dest = TokenizerWrap(texts=data_dest,
                               padding = 'post',
                               reverse=False,
                               num_words=num_words)

Yukarıdaki iki kod ile ingilizce ve türkçe cümlelerin bulunduğu verileri tokenleştirdik.

In [0]:
tokens_src = tokenizer_src.tokens_padded
tokens_dest = tokenizer_dest.tokens_padded
print(tokens_src.shape)
print(tokens_dest.shape)

(473035, 11)
(473035, 10)


Veri sayısı ingilizce(encoder) ve türkçe(decoder) için 473035 geldi.

İngilizce kelimelerin boyutu ise 11 olarak belirlenmiş. Bu 11 tane token olacak demektir. Türkçe kelimelerin boyutu ise 10 olarak gelmiştir. Türkçe sondan eklemeli bir dil olduğu için türkçe de ingilizceden daha az kelime vardır. bu yüzden boyut küçük olmuş.

In [0]:
token_start = tokenizer_dest.word_index[mark_start.strip()]
token_start

1

In [0]:
token_end = tokenizer_dest.word_index[mark_end.strip()]
token_end

2

Başlangıç tokeninin 1, bitiş tokeninin ise 2 olduğunu görüyoruz. Şimdi bazı örneklere bakalım.

In [0]:
idx = 200
tokens_src[idx]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0, 1368,   42],
      dtype=int32)

Her bir array'i, ona karşılık gelen kelimeye dönüştürerek orijinal metni yeniden oluşturabiliriz.

In [0]:
tokenizer_src.tokens_to_string(tokens_src[idx])

'deep how'

In [0]:
data_serc[idx]

'How deep?'

In [0]:
tokens_dest[idx]

array([   1,   10,   20, 1355,    2,    0,    0,    0,    0,    0],
      dtype=int32)

Burada da her bir array'i, ona karşılık gelen kelimeye dönüştürerek türkçe çevirisini görebiliriz.

In [0]:
tokenizer_dest.tokens_to_string(tokens_dest[idx])

'ssss ne kadar derin eeee'

In [0]:
data_dest[idx]

'ssss Ne kadar derin? eeee'

Tokenleştirme işlemi gerçekleşti şimdi encoder'a vereceğimiz inputu ve decoder'a vereceğimiz input, output'u ayarlayacağız. Outputu başlangıç tokeni olmadan vereceğiz.

Verileri sinir ağını eğitmek için kolayca hazırlayabiliriz.

In [0]:
encoder_input_data = tokens_src

In [0]:
decoder_input_data = tokens_dest[:,:-1]
decoder_input_data.shape

(473035, 9)

In [0]:
decoder_output_data = tokens_dest[:,1:]
decoder_output_data.shape

(473035, 9)

Hem input hem output aynı numpy arrayi içerisinde. Burada farklı yansımalarını aldık ve hafızadan yer kazandık.

In [0]:
idx=200
decoder_input_data[idx]

array([   1,   10,   20, 1355,    2,    0,    0,    0,    0], dtype=int32)

In [0]:
decoder_output_data[idx]

array([  10,   20, 1355,    2,    0,    0,    0,    0,    0], dtype=int32)

In [0]:
tokenizer_dest.tokens_to_string(decoder_input_data[idx])

'ssss ne kadar derin eeee'

In [0]:
tokenizer_dest.tokens_to_string(decoder_output_data[idx])

'ne kadar derin eeee'

İlk önce, bir tamsayı tokenleri dizisini bir "düşünce vektörüne" eşleyen sinir ağının kodlayıcı kısmını yaratıyoruz. Bunun için ilk olarak sinir ağının tüm katmanları için nesneleri oluşturduğumuz ve daha sonra bunları bağladığımız Keras'ın işlevsel API'sını kullanacağız, bu Keras'taki sıralı API'den daha fazla esneklik sağlar, daha karmaşık mimarileri denerken ve encoder ile decoderı bağlamanın yolları için faydalıdır.


In [0]:
encoder_input = Input(shape=(None,), name='encoder_input')

In [0]:
embedding_size=128

In [0]:
encoder_embedding = Embedding(input_dim=num_words,
                              output_dim=embedding_size,
                              name='encoder_embedding')

Model oluşturmaya encoder ile başladık.Keras için embedding layer oluşturuyoruz. ilk önce input_dim ile toplam kelime sayısını söylüyoruz. Output_dim ise vektör uzunluğudur.

In [0]:
state_size = 512 #bütün GRU layerlar 512 boyutlu output üretecek.

In [0]:
encoder_gru1 = GRU(state_size, name='encoder_gru1',
                   return_sequences=True)
encoder_gru2 = GRU(state_size, name='encoder_gru2',
                   return_sequences=True)
encoder_gru3 = GRU(state_size, name='encoder_gru3',
                   return_sequences=False)

Şimdi layerları bağlamamız gerekiyor.

In [0]:
def connect_encoder():
  net=encoder_input

  net=encoder_embedding(net)

  net=encoder_gru1(net)
  net=encoder_gru2(net)
  net=encoder_gru3(net)

  encoder_output=net

  return encoder_output


In [0]:
encoder_output = connect_encoder()

Önce embedding ürettik daha sonra 3 katlı gru kullanarak bir düşünce vektörü ürettik.

Şimdi decoder üretelim. Decoder'a başlangıç tokeniyle düşünce vektörünü vereceğiz ve model cümle üretecek. Bu cümlenin encoder'a verilen cümlenin çevirisi olmasını bekliyoruz.

In [0]:
decoder_initial_state = Input(shape=(state_size,),
                              name='decoder_initial_state')

In [0]:
decoder_input = Input(shape=(None,), name='decoder_input')

input_dim ile kelime haznemizdeki kelime sayısını belirttik. Output_dim ise vektör uzunluğudur. şimdilik kelime vektörlerini rastgele sayılardan oluşturduk ancak eğitim esnasında bu vektörler optimize edilecek.

In [0]:
decoder_embedding= Embedding(input_dim=num_words,
                             output_dim=embedding_size,
                             name='decoder_embedding')

In [0]:
decoder_gru1 = GRU(state_size, name='decoder_gru1',
                   return_sequences=True)
decoder_gru2 = GRU(state_size, name='decoder_gru2',
                   return_sequences=True)
decoder_gru3 = GRU(state_size, name='decoder_gru3',
                   return_sequences=True)

Cümle üretmesini istediğimiz için son return_sequences'i true olarak bıraktık. Böylece input, embedding, GRU layerlarını oluşturduk. Decoder'ın çalışması üzerine kelimeler elde etmek istiyoruz. Şu an 512 tane vektör elde ediyoruz ve bunu kelimelere dönüştürmeliyiz. Bunu yapmak için dense oluşturacağız. Gru sonucunu one hot türünde  array'a dönüştüreceğiz. Bu vektörde en büyük değere sahip elemanın indeksi output olarak hangi kelimenin verilmesi gerektiğini belirleyecek.

In [0]:
decoder_dense = Dense(num_words,
                      activation='softmax',
                      name='decoder_output')

In [0]:
def connect_decoder(initial_state):
  net=decoder_input

  net=decoder_embedding(net)
  
  net=decoder_gru1(net,initial_state=initial_state)
  net=decoder_gru2(net,initial_state=initial_state)
  net=decoder_gru3(net,initial_state=initial_state)

  decoder_output = decoder_dense(net)

  return decoder_output

İnitial_state encoder'ın ürettiği düşünce vektörüdür. Bir önceki layerı parantez içinde vererek layerları bağlamış oluyoruz. Encoder ve decoder'ı birbirine bağlayıp uçtan uca modeli eğitelim. 

In [0]:
decoder_output = connect_decoder(initial_state=encoder_output)

model_train = Model(inputs=[encoder_input,decoder_input],
                    outputs=[decoder_output])

Eğitebileceğimiz bir model oldu. encoder ve decoder'ı birbirine bağlamış bulunuyoruz. İki model daha üreteceğiz (biri encoder diğeri decoder için) bunlar çeviri yapmamızda işimize yarayacak. 

In [0]:
model_encoder = Model(inputs=[encoder_input],
                      outputs=[encoder_output])

In [0]:
decoder_output = connect_decoder(initial_state = decoder_initial_state)

model_decoder = Model(inputs=[decoder_input, decoder_initial_state], 
                      outputs=[decoder_output])

Oluşturduğumuz ilk model ile sinir ağımızı eğiteceğiz diğer iki modelle ise eğitimi tamamladıktan sonra çeviri yapmak için kullanacağız. Gerçek değerlerin bulunduğu one hot array oluşturacağız.

In [0]:
model_train.compile(optimizer = RMSprop(lr=1e-3),
                    loss='sparse_categorical_crossentropy')

Burada yer alan loss fonk ile tahmin edilen değerle gerçekte olması gereken değer karşılaştırılacak ve hatanın büyüklüğünü belirten değer gelecektir. Eğitim yapabilmek için loss değerine ihtiyacımız var.

RDD'lerde iyi çalıştığı için optimizer için RMSprop kullanıyoruz.

In [0]:
x_data = \
{
    'encoder_input': encoder_input_data,
    'decoder_input': decoder_input_data
}

In [0]:
y_data = \
{
    'decoder_output': decoder_output_data
}

In [0]:
validation_split = 10000 / len(encoder_input_data)
validation_split

0.021140084771739936

In [0]:
model_train.fit(x=x_data,
                y=y_data,
                batch_size=256,
                epochs=1,
                validation_split=validation_split)

Train on 463035 samples, validate on 10000 samples
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
463035/463035 [==============================] - 10101s 22ms/sample - loss: 2.3690 - val_loss: 4.8115


Modeli eğitiyoruz ve inputları, outputları veriyoruz. Her iterasyonda 256 veri veriyoruz ve her seferinde o veriyi eğitiyoruz. Verilerin üzerinden 1 defa eğitim gerçekleşecektir. Bu kod çalıştırıldığında model eğitilmeye başlayacaktır. Ekran kartınızın ram'ı daha fazlaysa epochs değerinizi arttırabilirsiniz. Daha iyi sonuçlar alırsınız.

In [0]:
def translate(input_text, true_output_text=None):
  #input için tokenleştirme yapıyoruz.
  input_tokens=tokenizer_src.text_to_tokens(text=input_text,
                                             reverse=True,
                                             padding='pre')
  #düşünce vektörü üretiyoruz.
  initial_state = model_encoder.predict(input_tokens)
  #decoder için max. token sayısı kadar döngü dönecek
  max_tokens= tokenizer_dest.max_tokens

  shape = (1,max_tokens)
  decoder_input_data= np.zeros(shape=shape, dtype=np.int)

  token_int=token_start #başlangıç tokeni yani ssss
  output_text='' 
  count_tokens=0 #döngünün kaç defa döndüğünü sayacağız.

  while token_int != token_end and count_tokens < max_tokens:
    decoder_input_data[0, count_tokens] = token_int
    x_data = \
    {
        'decoder_initial_state':initial_state,
        'decoder_input': decoder_input_data
    }

    decoder_output = model_decoder.predict(x_data) 

    token_onehot = decoder_output[0, count_tokens, :] 
    token_int = np.argmax(token_onehot) #en yüksek indexe sahip elemanı elde ediyoruz.

    sampled_word = tokenizer_dest.token_to_word(token_int) #burada ise yüksek indexe sahip kelimeyi yazdırıyoruz.
    output_text += " " + sampled_word
    count_tokens += 1 

  output_tokens = decoder_input_data[0] #üretilen cümleyi yazdıralım.

  print("input text:")
  print(input_text)
  print()

  print("translated text:")
  print(output_text)
  print()

  if true_output_text is not None:
    print("true output text:")
    print(true_output_text)
    print()



Eğitilmiş modeli kullanarak çeviriyi yapalım. İngilizce cümleyi model encoder'a vereceğiz ve düşünce vektörü elde edeceğiz bunu ve başlangıç tokenini model decoder'a vereceğiz. Tamamen türkçe cümle vermiyoruz böylece modelin cümle üretmesini sağlıyoruz.

Modelin çevirisini ve doğru çeviriyi karşılaştırıyoruz.

In [0]:
translate(input_text=data_serc[10000],
          true_output_text=data_dest[10000])

input text:
I like it, too.

translated text:
 ben de onu seviyorum eeee

true output text:
ssss Onu ben de beğeniyorum. eeee



In [0]:
translate(input_text=data_serc[5000],
          true_output_text=data_dest[5000])

input text:
Tom seems OK.

translated text:
 tom iyi görünüyor eeee

true output text:
ssss Tom iyi görünüyor. eeee



In [0]:
translate(input_text=data_serc[70000],
          true_output_text=data_dest[70000])

input text:
That's totally wrong.

translated text:
 bu tamamen yanlış eeee

true output text:
ssss Bu tamamen yanlış. eeee



In [0]:
translate(input_text='What is your name?')

input text:
What is your name?

translated text:
 adı nedir eeee



Bu cümle veri setinde bulunmuyor. Model cümleyi iyi çevirdi.

Kaynaklar;

https://www.tensorflow.org/

https://github.com/Hvass-Labs/TensorFlow-Tutorials

https://www.youtube.com/results?search_query=neural+machine+translation
